# 📥 01 – Data Acquisition

This notebook pulls EM equity and macroeconomic time series data from Bloomberg using the BQL API.

In [ ]:

import pandas as pd
import bql
import os

bq = bql.Service()
date_range = bq.func.range('-1Y', '0D')

em_assets = {
    'Brazil_EWZ': 'EWZ US Equity',
    'India_INDA': 'INDA US Equity',
    'China_FXI': 'FXI US Equity',
    'SouthAfrica_EZA': 'EZA US Equity',
    'Mexico_EWW': 'EWW US Equity',
    'Indonesia_EIDO': 'EIDO US Equity'
}

em_data = {}
for label, ticker in em_assets.items():
    data_item = bq.data.px_last(dates=date_range, fill='prev')
    request = bql.Request(ticker, data_item)
    response = bq.execute(request)
    df = response[0].df()
    px_col = [col for col in df.columns if 'PX_LAST' in col.upper()][0]
    df = df[['DATE', px_col]]
    df.columns = ['date', label]
    df.set_index('date', inplace=True)
    em_data[label] = df

em_df = pd.concat(em_data.values(), axis=1)

macro_assets = {
    'USD_Index': 'DXY Curncy',
    'Oil_Brent': 'CO1 Comdty',
    'US_10Y_Yield': 'USGG10YR Index',
    'Fed_Funds': 'FDTR Index',
    'VIX': 'VIX Index',
    'Copper': 'LMCADY Comdty'
}

macro_data = {}
for label, ticker in macro_assets.items():
    data_item = bq.data.px_last(dates=date_range, fill='prev')
    request = bql.Request(ticker, data_item)
    response = bq.execute(request)
    df = response[0].df()
    px_col = [col for col in df.columns if 'PX_LAST' in col.upper()][0]
    df = df[['DATE', px_col]]
    df.columns = ['date', label]
    df.set_index('date', inplace=True)
    macro_data[label] = df

macro_df = pd.concat(macro_data.values(), axis=1)
combined_df = pd.merge(em_df, macro_df, left_index=True, right_index=True)
combined_df = combined_df.sort_index().dropna()

os.makedirs('../data', exist_ok=True)
combined_df.to_csv('../data/combined_em_macro_data.csv')
combined_df.head()
